# Will it Rain Tomorrow?
### Deep Learning with Neural Networks and RNNs

Predict whether it rains

In [66]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle

# workaround for MacOS/jupyter notebook bug w/ tensorflow
# https://www.programmersought.com/article/69923598438/
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Import Data

In [58]:
np.random.seed(4471)
# https://www.tensorflow.org/tutorials/load_data/numpy
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
weather_pd = pd.read_csv('../data/weather.csv', index_col = 0)
weather_pd = weather_pd.drop(['DAY', 'STP', 'GUST'], axis=1)
weather_np = weather_pd.to_numpy()

In [59]:
weather_pd.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0


In [60]:
# whether it rained that day
# bool to int conversion: https://stackoverflow.com/questions/17506163/how-to-convert-a-boolean-array-to-an-int-array
weather_pd['RAIN'] = (weather_pd['PRCP'] > 0).astype(int)

In [61]:
weather_pd.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP,RAIN
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0,0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0,0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0,0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0,0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0,1


In [62]:
weather_pd.dtypes

YEAR        int64
MONTH       int64
SEASON      int64
TEMP      float64
DEWP      float64
SLP       float64
VISIB     float64
WDSP      float64
MXSPD     float64
MAX       float64
MIN       float64
PRCP      float64
SNDP      float64
RAIN        int64
dtype: object

## Divide data into test/train

In [67]:
# adapted from original code in project 2
def divide_data(weather):
    '''divide dataset into two sets: 90% train and 10% test'''
    n = weather.shape[0]
    
    # shuffle data for test/train so no patterns
    # https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    weather = shuffle(weather)
    
    # take out 10% of the data for validation
    # https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html
    ind_test = np.random.choice(n, size = n // 10, replace = False)
    weather_test = weather.iloc[ind_test]

    # take the other 90% for building the model
    # https://stackoverflow.com/questions/27824075/accessing-numpy-array-elements-not-in-a-given-index-list
    ind_train = [x for x in range(n) if x not in ind_test] # not in index
    weather_train = weather.iloc[ind_train]

    return weather_test, weather_train

In [70]:
weather_test, weather_train = divide_data(weather_pd)
print("test: ", weather_test.shape)
print("train:", weather_train.shape)
weather_train.head()

test:  (777, 14)
train: (6995, 14)


,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP,RAIN
1017,2002,10,3,59.8,52.0,1023.0,9.2,8.0,13.0,73.9,55.0,0.08,0.0,1
2968,2008,2,0,50.4,33.9,1022.8,10.0,5.5,9.9,61.0,33.1,0.00,0.0,0
7554,2020,9,2,69.4,56.7,1022.4,10.0,3.3,8.9,82.0,57.0,0.00,0.0,0
3570,2009,10,3,76.0,67.2,1013.0,9.7,7.9,14.0,82.9,55.9,0.00,0.0,0
1131,2003,2,0,43.2,16.7,1017.2,9.9,5.4,12.0,63.0,33.1,0.21,0.0,1


## Divide Each Set into Features/Targets

In [74]:
def separate_targets(weather):
    '''separate dataset into features and targets'''
    # target: whether next day rains
    target = weather[['RAIN']].iloc[1:, :]
    target = np.round(target.to_numpy().reshape(-1))

    # feature: today's weather (array of 13 vars)
    feature = weather.iloc[:-1, :-1].to_numpy()
    
    return feature, target

In [78]:
feature_test, target_test = separate_targets(weather_test)
feature_train, target_train = separate_targets(weather_train)

print("feature:", feature_test.shape, "target:", target_test.shape)
print("feature:", feature_train.shape, "target:", target_train.shape)

feature: (776, 13) target: (776,)
feature: (6994, 13) target: (6994,)


## Build the Model

In [154]:
# how many possible outputs
num_output_vals = len(np.unique(target_test))

# num nodes in first layer
first_layer = 64

In [155]:
def build_model(first_layer, num_output_vals):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(first_layer, activation='sigmoid', dtype='float64'),
        tf.keras.layers.Dropout(0.2, dtype='float64'),
        tf.keras.layers.Dense(num_output_vals, activation='sigmoid', dtype='float64')
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [156]:
model = build_model(first_layer, num_output_vals)

## Example model output

In [82]:
example = model(feature_train[:1, :])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [83]:
example.shape # good

TensorShape([1, 2])

In [84]:
example

<tf.Tensor: id=49375, shape=(1, 2), dtype=float64, numpy=array([[0.3279861 , 0.22379077]])>

## Train the model

In [85]:
EPOCHS = 30

In [86]:
history = model.fit(feature_train, target_train, epochs=EPOCHS)

Train on 6994 samples
Epoch 1/30
6994/6994 [==============================] - 1s 127us/sample - loss: 0.6770 - accuracy: 0.6100
Epoch 2/30
6994/6994 [==============================] - 0s 62us/sample - loss: 0.6646 - accuracy: 0.6263
Epoch 3/30
6994/6994 [==============================] - 0s 61us/sample - loss: 0.6630 - accuracy: 0.6354
Epoch 4/30
6994/6994 [==============================] - 0s 58us/sample - loss: 0.6584 - accuracy: 0.6361
Epoch 5/30
6994/6994 [==============================] - 0s 56us/sample - loss: 0.6593 - accuracy: 0.6365
Epoch 6/30
6994/6994 [==============================] - 0s 57us/sample - loss: 0.6588 - accuracy: 0.6365
Epoch 7/30
6994/6994 [==============================] - 0s 61us/sample - loss: 0.6580 - accuracy: 0.6367
Epoch 8/30
6994/6994 [==============================] - 0s 58us/sample - loss: 0.6576 - accuracy: 0.6367
Epoch 9/30
6994/6994 [==============================] - 0s 61us/sample - loss: 0.6577 - accuracy: 0.6367
Epoch 10/30
6994/6994 [=========

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              multiple                  896       
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_9 (Dense)              multiple                  130       
Total params: 1,026
Trainable params: 1,026
Non-trainable params: 0
_________________________________________________________________


## Predict for example

In [88]:
pred = model.predict(feature_test[88].reshape(1,13))
print(sum(sum(pred)))
print(pred.argmax())

1.5379024119201072
0


In [ ]:
# find in original data - see if it rained that day

## Evaluate on Test Set

In [89]:
model.evaluate(feature_test, target_test, verbose=2)

776/1 - 0s - loss: 0.6258 - accuracy: 0.6405


[0.6536884865502721, 0.6404639]

not that accurate - reasonable because has not sense of trends

## Predict Rain Over Time

for given 6 days of rain, will it rain the 7th?

In [98]:
rain = weather_pd[['RAIN']].to_numpy().reshape(-1)

In [103]:
rain.shape

array([0, 0, 0, ..., 1, 0, 0])

## Create Features and Targets

In [157]:
# features are 6 days
# targets are 6 days shifted over 1
# see shakespeare

# The maximum length sentence we want for a single input in characters
seq_length = 6
examples_per_epoch = len(rain)//(seq_length+1)

# Create training examples / targets
rain_tf = tf.data.Dataset.from_tensor_slices(rain)

for i in rain_tf.take(5):
  print(i.numpy())

0
0
0
0
1


In [158]:
weeks = rain_tf.batch(seq_length+1, drop_remainder=True)

for day in weeks.take(5):
  print(day.numpy())

[0 0 0 0 1 0 1]
[1 1 1 1 0 0 0]
[0 0 0 1 1 1 0]
[0 1 1 1 1 0 0]
[0 1 1 0 0 0 0]


In [159]:
def split_input_target(week):
    '''duplicate and shift weeks to form input and target days'''
    input_days = week[:-1]
    target_days = week[1:]
    return input_days, target_days

dataset = weeks.map(split_input_target)

In [160]:
for input_example, target_example in  dataset.take(3):
    print ('Input data: ', input_example.numpy())
    print ('Target data:', target_example.numpy())

Input data:  [0 0 0 0 1 0]
Target data: [0 0 0 1 0 1]
Input data:  [1 1 1 1 0 0]
Target data: [1 1 1 0 0 0]
Input data:  [0 0 0 1 1 1]
Target data: [0 0 1 1 1 0]


In [161]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input:", input_idx.numpy())
    print("  expected output:", target_idx.numpy())

Step    0
  input: 0
  expected output: 0
Step    1
  input: 0
  expected output: 0
Step    2
  input: 0
  expected output: 1
Step    3
  input: 1
  expected output: 1
Step    4
  input: 1
  expected output: 1


## Create training batches

In [162]:
# Batch size
BATCH_SIZE = 4

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((4, 6), (4, 6)), types: (tf.int64, tf.int64)>

## Build the Model

In [163]:
# how many possible outputs
num_output_vals = len(np.unique(rain))

# embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 16

In [168]:
def build_model(num_output_vals, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(num_output_vals, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(num_output_vals)
    ])
    return model

In [165]:
#def build_model(rnn_units, num_output_vals, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, batch_input_shape = (batch_size, 6)),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(num_output_vals)
    ])
    return model

IndentationError: unexpected indent (<ipython-input-165-8aec427b9cb9>, line 2)

In [169]:
model = build_model(num_output_vals, embedding_dim, rnn_units, BATCH_SIZE)

In [167]:
# set floats to float64 to avoid warning message
# tf.keras.backend.set_floatx('float64')

## Try the Model

In [170]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, num_output_vals)")

(4, 6, 2) # (batch_size, sequence_length, num_output_vals)


In [171]:
target_example_batch.shape

TensorShape([4, 6])

In [172]:
input_example_batch.shape

TensorShape([4, 6])

In [173]:
example_batch_predictions.shape

TensorShape([4, 6, 2])

In [174]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (4, None, 256)            512       
_________________________________________________________________
gru_3 (GRU)                  (4, None, 16)             13152     
_________________________________________________________________
dense_16 (Dense)             (4, None, 2)              34        
Total params: 13,698
Trainable params: 13,698
Non-trainable params: 0
_________________________________________________________________


In [175]:
# example predictions, not accurate bc model not trained
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([0, 1, 1, 1, 1, 1])

In [176]:
# show input and output
print("Input: \n", input_example_batch[0].numpy())
print()
print("Next Day Predictions: \n", sampled_indices)

Input: 
 [1 0 1 1 0 0]

Next Day Predictions: 
 [0 1 1 1 1 1]


In [177]:
# set up loss function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, days_in_batches, num_output_vals)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (4, 6, 2)  # (batch_size, days_in_batches, num_output_vals)
Mean loss:         0.7077194


In [178]:
# exponential of the mean loss should ~= num outputs
tf.exp(mean_loss).numpy()

2.029358

In [179]:
# set optimize and loss fucntion
model.compile(optimizer='adam', loss=loss)

## Configure Checkpoints
save checkpoints during training

In [180]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training-checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Train Model

In [181]:
EPOCHS = 30

In [182]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
277/277 [==============================] - 5s 16ms/step - loss: 0.6214
Epoch 2/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6155
Epoch 3/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6146
Epoch 4/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6135
Epoch 5/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6139
Epoch 6/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6131
Epoch 7/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6131
Epoch 8/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6137
Epoch 9/30
277/277 [==============================] - 2s 9ms/step - loss: 0.6155
Epoch 10/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6140
Epoch 11/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6145
Epoch 12/30
277/277 [==============================] - 2s 8ms/step - loss: 0.6134
Epoch 13/30
277/277 [===

## Restore Latest Checkpoint

In [184]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(num_output_vals, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            512       
_________________________________________________________________
gru_4 (GRU)                  (1, None, 16)             13152     
_________________________________________________________________
dense_17 (Dense)             (1, None, 2)              34        
Total params: 13,698
Trainable params: 13,698
Non-trainable params: 0
_________________________________________________________________


In [193]:
example_week = input_example_batch[:1]
example_week.shape

TensorShape([1, 6])

In [194]:
model.reset_states()
test = example_week[:1]
predictions = model(test)
predictions = tf.squeeze(predictions, 0)
predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
test = tf.expand_dims([predicted_id], 0)
print(predictions.shape)
print(predicted_id)
test

(6, 2)
0


<tf.Tensor: id=103586, shape=(1, 1), dtype=int32, numpy=array([[0]], dtype=int32)>

In [197]:
def generate_weather(model, start_weather):
    # Evaluation step (generating weather using the learned model)

    # Number of days to generate
    num_generate = 8

    # Empty string to store our results
    days_generated = []

    # Low temperatures results in more predictable results.
    # Higher temperatures results in more surprising results.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(start_weather)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the temperature returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # we pass the predicted weather as the next input to the model
        # along with the previous hidden state
        start_weather = tf.expand_dims([predicted_id], 0)

        days_generated.append(predicted_id)

    return days_generated

In [204]:
print("start days:         ", list(example_week.numpy()[0]))
print("predicted next days:", generate_weather(model, start_weather=example_week))

start days:          [1, 0, 1, 1, 0, 0]
predicted next days: [0, 0, 0, 0, 0, 0, 0, 1]


In [205]:
## locate week to see if that was right?


#### Resources Consulted
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf